# Exercise 1 - Basic prompt engineering
The goal of this exercise is to become familiar with the basic prompt engineering techniques.
We do this by first interactively exploring the examples from the slides.
Then, we will apply the same techniques to extract information from a dataset of housing descriptions.



To do this will make use of the cloud-agnostic **LangChain** API.

## Introduction to LangChain

In the previous chapter, we explored how to interact directly with language models, like OpenAI's GPT or Google's Gemini (Vertex AI). While these APIs are powerful, working with them directly can become repetitive and cloud-specific.

LangChain is a Python-based framework that simplifies the use of language models, providing a cloud-agnostic interface for interacting with providers like Google Vertex AI and (Azure) OpenAI. With LangChain, you can focus on what you want the model to do, not how to interact with different APIs.

**Why Use LangChain?**
- Cloud-Agnostic: Switch between providers (e.g., OpenAI, Azure, Google) with minimal changes to your code.
- Simplified Usage: LangChain abstracts away API-specific details, letting you use models in a more consistent and straightforward way.
- Scalability: As your projects grow, LangChain makes it easier to manage interactions with models.



Let's import the necessary libraries and create a client for our LLM.

In [75]:
from llm_in_production.llm import instantiate_langchain_model
import dotenv
import pandas as pd


dotenv.load_dotenv()

# Here we create the client. 
# Make sure you select the LLM provider that corresponds to the one you are using in this course!
client = instantiate_langchain_model(
    # llm_provider="azure",
    llm_provider="gcp",
)
client.model_name

'gemini-1.5-flash'

## Exercise 1a: Examples from the slides
In this exercise, you can interactively re-run the examples from the slides.


For each example, do the following:
- First, run the example as is and observe the output.
- Compare how using the LangChain model differs from using the cloud-specific API.
- Next, modify the prompt and see how it affects the output.
- Optionally, try giving it a different problem to see how it handles that as well.


### Classification

It is possible to write prompts that transform LLMs into classifers.


In [2]:
system_prompt = "Classify the user messages into neutral, negative, or positive. Respond only with one of the above classes."
text = "I think the food was okay."

messages = [
    ("system", system_prompt),
    ("human", text),
]
response = client.invoke(messages)
response.content

'Neutral \n'

The prompt above uses a chat-style format, where a system message defines the task (classifying sentiment) and a human message provides the input text, mimicking a conversational interaction with the language model.

### Summarization

We can also use the LLM to summarize text. Here we show an example of summarization prompt.

In [3]:
text = """
Extract the names of locations in the following text.
Desired format:
Places: Comma separated list of location_names
Input: In Paris, love's embrace ignites the night, Venice's canals whisper secrets in moonlight, Berlin's walls echo stories of resilience and might
Places: """

response = client.invoke(
    text
)
completion = response.content.strip()
places = completion.split(",")
print(places)

['Places: Paris', ' Venice', ' Berlin']


The above eprompt uses an instruction-following format, where a clear instruction and desired output format are provided within the input text. The language model processes the input and generates a response in the specified format, extracting location names from the given text.

### Chain of thought reasoning
For more complex reasoning tasks, we can use the [chain of thought reasoning technique](https://www.promptingguide.ai/techniques/cot#chain-of-thought-cot-prompting). 

With this style, the LLM first generates a thought by writing down its observations, reasoning, and calculations. Then, it gives as final answer.

Using the instruction-following format, the prompt could looks as follows:

In [4]:
instruction_prompt = """
You solve mathematical problems by writing down your reasoning and calculations and then writing down the answer.

Respond using this format:
Thought: describe here you write reasoning, and calculations need to solve the problem
Answer: your answer without any explanation, text or calculations.


Problem: I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1.
Thought:""".strip()


Whilst using the chat-style format could look like this:

In [5]:
system_prompt = """
The user will give you mathematical problems to solve.
You should solve the problems and respond with the answer.
Respond using this format:
Thought: describe here you write reasoning, and calculations need to solve the problem
Answer: your answer without any explanation, text or calculations.
""".strip()

user_message = """
I went to the market and bought 10 apples. 
I gave 2 apples to the neighbor and 2 to the repairman. 
I then went and bought 5 more apples and ate 1.
How many apples did I remain with?
""".strip()

chat_prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message},
    ]

Compare the responses when using the instruction-following and chat-style prompting.

In [6]:

response = client.invoke(
    # instruction_prompt,
    chat_prompt
)
completion = response.content
print(completion)
answer = completion.split("Answer:")[1].strip()
print("#" * 80)
print("Extracted answer:", answer)

Thought: Start with 10 apples, subtract the 2 given to the neighbor and 2 given to the repairman. Then add the 5 purchased and subtract the 1 eaten. 

Answer: 11 

################################################################################
Extracted answer: 11


### Few shot examples
Few shot examples are another useful technique to guide the LLM to generate a specific output.
It is especially useful when it easier to just show the desired format, instead of describing it.

This way of prompting suits the chat-style format.

In [7]:
response = client.invoke(
    input=[
        {"role": "system", "content": "Posible values for sentiment: neutral, negative, good"},
        {"role": "user", "content": "I think the food was okay."},
        {"role": "assistant", "content": "Sentiment: neutral"},
        {"role": "user", "content": "I think the food was bad."},
        {"role": "assistant", "content": "Sentiment: negative"},
        {"role": "user", "content": "I think the food was good."},
    ],
    temperature=0.0,
)
completion = response.content.strip()
print(completion)

Sentiment: good


## Extracting housing information
Now that we have seen some examples of prompt engineering, let's apply them to extract information from a dataset of housing descriptions. 

We have a dataset with house listing descriptions. This description contains information about the house, such as the number of bedrooms, the neighborhood, and whether pets are allowed. However, everybody writes the information slightly differently, making it hard to parse it using regular expressions. Let's try to use the LLM to extract this information.

We have also included the known answer for each example in the dataset. This makes it easier to evaluate your solution, although in practice, we would not have this information.


In [8]:
df = pd.read_csv("houses.csv")
print(df.shape)
df

(10, 8)


,description,city,price,surface_area,neighborhood,bedrooms,house_type,pets_allowed
0,This tastefully styled one-bedroom apartment i...,Amsterdam,2350,67,Da Costabuurt,1,Appartement,allowed
1,Amazing double floor house with 2 balconys (11...,Amsterdam,3250,119,Oud-Zuid,3,Appartement,unknown
2,Details:\n- Size of the property: 78 m2\n- Num...,Amsterdam,2500,78,Kinkerbuurt,2,Appartement,not_allowed
3,"On this beautiful location in the Geuzenbuurt,...",Amsterdam,2000,56,Geuzenbuurt,2,Appartement,not_allowed
4,Viewings; reequest per e-mail only. Not for st...,Amsterdam,2150,58,De Pijp,1,Appartement,unknown
5,Apartment on 1 minute walking distance of the ...,Amsterdam,1900,70,Nieuw-Sloten,2,Appartement,not_allowed
6,THIS FURNISHED ONE BEDROOM APARTMENT IS LOCATE...,Amsterdam,2050,50,Helmersbuurt,1,Appartement,unknown
7,— ENGLISH TRANSLATION BELOW —\n AANVRAGEN VOOR...,Amsterdam,2400,63,De Pijp,2,Appartement,unknown
8,"""This very spacious unfurnished apartment with...",Amsterdam,5000,225,Apollobuurt,6,Appartement,unknown
9,"We proudly offer For Rent this spacious, luxur...",Amsterdam,2350,85,Buitenveldert-West,2,Appartement,unknown


As you can see, we have the following information available:

In [54]:
house_idx =1
row = df.iloc[house_idx]
print("House:", house_idx)
print("City:", row["city"])
print("Price:", row["price"])
print("Surface area:", row["surface_area"])
print("Bedrooms:", row["bedrooms"])
print("Description: ", row["description"])
print("Neigborhood: ", row["neighborhood"])

IndexError: single positional indexer is out-of-bounds

## Exercise 1b: Extract the number of bedrooms
The first thing we want to extract is the number of bedrooms.
This should be a positive integer number.
It is your task to write a prompt that extracts this information from the description in the function below.

For example:
- Description: "The house has 3 bedrooms and two bathrooms."
- Extracted number: "3"

If you run this cell, it will automatically test your function against the known correct answers in the dataset.

In [90]:
def extract_n_bedrooms(description: str) -> str:
    # YOUR CODE HERE START
    system_prompt="Extract ONLY the number of bedrooms in the following text. Your response will be only number or digit! A house can have different types of rooms and we are interested only in the number of bedrooms. Only the bedroom, like 1 bedroom or one-bedroom meaning 1."
    # YOUR CODE HERE END
    
    # Feel free to change more of the code below if you want to
    # This is just a skeleton to get you started
    response = client.invoke(
        input=[
            {"role": "system", "content": system_prompt},
            # YOUR CODE HERE START: Optionally add some few shot examples here
            {"role": "user", "content": "The house has 4 rooms: 1 dining room, 1 family room, and 2 bedrooms."},
            {"role": "system", "content": "2"},
            {"role": "user", "content": "This tastefully styled one-bedroom apartment is located in the heart of Amsterdam Oud-West in the Da Costabuurt neighborhood. This beautiful home in the bustling center of Amsterdam tucked away on a quiet street can be your home away from home"},
            {"role": "system", "content": "1"},
            {"role": "user", "content": """Amazing double floor house with 2 balconys (119m2) attached to the canal. Oud-Zuid is a desirable area. It always has been and always will be. Countless monumental buildings, bustling streets, cozy restaurants and trendy shops (Beethovenstraat, Marathonweg en Stadionplein) contribute to the area's unique scene. Sophisticated, but very "now". Just around the corner from this street, the entire world is at your fingertips! Private entrance; Very spacious and sun filled living overlooking the Canal; Separate and modern kitchen equipped with all built-in appliances; Master bedroom with double bed and design built in wardrobes Beautiful bathroom, with walk-in shower, double sink and toilet; 2nd (Guest) toilet; large basement for extra storage Separate laundry area (including washer and dryer); 2nd and 3rd bedroom are both very well sized.""" },
            {"role": "system", "content": "3"},
            # YOUR CODE HERE END
            {"role": "user", "content": description},
        ],
        temperature=0.2,
    )
    return response.content.strip()


# Here we test your function against the known correct answers
# Feel free to inspect df["bedrooms"] to get a better understanding go the answers.
for i, row in df.iterrows():
    n_bedrooms = extract_n_bedrooms(row["description"])
    assert n_bedrooms.isdigit() or isinstance(n_bedrooms, float), f"For row {i}, we got not a number: {n_bedrooms}"
    assert row["bedrooms"] == int(n_bedrooms), f"For row {i}, we expected {row['bedrooms']} but got {n_bedrooms}"
    print(f"✅ Row {i} passed `{n_bedrooms}` == `{row['bedrooms']}`")

AssertionError: For row 0, we got not a number: 2
1
1
2
1
1

## Exercise 1c: Extract neighborhood name
The second thing we want to extract is the name of the neighborhood.
This should be a string with the name of the neighborhood.
It is your task to write a prompt that extracts this information from the description in the function below.

For example:
- Description: "The house is located in De Pijp."
- Extracted neighborhood: "De Pijp"

If you run this cell, it will automatically test your function against the known correct answers in the dataset.

In [84]:
def extract_neighborhood(description):
    # YOUR CODE HERE START
    system_prompt="Extract ONLY the neighborhood or of a house where the house is located given a house description. Each house has only has one neigbourhood. The format should only the neigborhood name. Neighborhood is not the same as city and neighborhood can belong to bigger neighborhoods, tried to get the most specific one and only that one. Usually it starts with located or followed by neighborhood. It is an area in the smallest part of the city usually one or two words length."
    # YOUR CODE HERE END
    
    # Feel free to change more of the code below if you want to
    # This is just a skeleton to get you started
    response = client.invoke(
        input=[
            {"role": "system", "content": system_prompt},
             # YOUR CODE HERE START: Optionally add some few shot examples here
            {"role": "user", "content":  "It is located in the heart of Amsterdam Oud-West in the Da Costabuurt neighborhood. This beautiful home in the bustling center of Amsterdam tucked away on a quiet street can be your home away from home." },
            {"role": "system", "content": "Da Costabuurt"},
            {"role": "user", "content": "Amazing double floor house in Oud-Zuid is a desirable area." },
            {"role": "system", "content": "Oud-Zuid"},
            {"role": "user", "content":  "On this beautiful location in the Geuzenbuurt, situated between Oud-West and de Baarsjes and conveniently close to the city center, you'll find this stunning 2 bedroom apartment measuring approximately 56 m2, with a balcony and storage. The apartment is located on the most beautiful and wide section of Willem de Zwijgerlaan" },
            {"role": "system", "content": "Geuzenbuurt"},
            # YOUR CODE HERE END
            {"role": "user", "content": description},

        ],
        temperature=0.0,
    )
    message = response.content
    
    return message.replace(".", "").strip()

# Here we test your function against the known correct answers
# Feel free to inspect df["neighborhood"] to get a better understanding go the answers.
for i, row in df.iterrows():
    neighborhood_name = extract_neighborhood(row["description"])
    assert neighborhood_name.lower().strip() == row["neighborhood"].lower().strip(), f"For row {i}, we expected `{row['neighborhood']}` but got `{neighborhood_name}`"
    print(f"✅ Row {i} passed {neighborhood_name} == {row['neighborhood']}")
    

AssertionError: For row 0, we expected `Da Costabuurt` but got `Da Costabuurt
Oud-Zuid
Geuzenbuurt 
Da Costabuurt 
Oud West`

## Exercise 1d: Extract if pets are allowed
The last thing we want to extract is whether pets are allowed.
This can be one of the following classes:
- `allowed`: If the description explicitly mentions that pets are allowed.
- `not_allowed`: If the description explicitly mentions that pets are not allowed.
- `unknown`: If the description does not explicitly mention whether pets are allowed or not.

It is your task to write a prompt that extracts this information from the description in the function below.

For example:
- Description: "No dogs allowed."
- Extracted: "not_allowed"

If you run this cell, it will automatically test your function against the known correct answers in the dataset.

In [76]:
def are_pets_allowed(description):
    # YOUR CODE HERE START
    system_prompt="Extract if the house allows to bring pets like cat or dog. The output should be either allowed, not_allowed, or unknown. If there is no information, just say unknown."
    # YOUR CODE HERE END

    response = client.invoke(
        input=[
            {"role": "system", "content": system_prompt},
            # YOUR CODE HERE START: Optionally, add some few shot examples here
            { "role": "user", "content": """Conditions:
- Pets not allowed
- Sharing not allowed
- Students not allowed
- Smoking not allowed
- Tenancy agreements subject to owner’s consent
- Measurements not conform NEN 2580
The rental price of this house is exclusive Gas/Electricity/Water, TV/Internet and local Taxes.",Amsterdam,2500,78,Kinkerbuurt,2,Appartement,not_allowed
"On this beautiful location in the Geuzenbuurt, situated between Oud-West and de Baarsjes and conveniently close to the city center, you'll find this stunning 2 bedroom apartment measuring approximately 56 m2, with a balcony and storage. The apartment is located on the most beautiful and wide section of Willem de Zwijgerlaan, offering unobstructed views of the magnificent chestnut trees lining the street and the recently architect-designed courtyard.
The apartment has a very practical layout with 2 bedrooms, a separate toilet, a spacious bathroom with a walk-in shower with a glass door and a sink, a spacious modern kitchen equipped with all built-in appliances and a composite countertop. The living room features high ceilings and plenty of natural light, and it adjoins the second bedroom through a beautiful design (steel glass) door. The east-facing balcony (morning sun) spans the entire width of the apartment and overlooks the beautiful courtyard. The garden, just recently landscaped by an architect, is for the exclusive use of the residents and they also have the option to store a bicycle/motorcycle/scooter under the covered area. Additionally, there is a convenient private storage space in the basement.
The apartment is part of Geuzenhof I, an unique residential complex recognized as a municipal monument. An active owners association ensures proper maintenance.
The location is""" },
            {"role": "system", "content": "not_allowed"},
            # YOUR CODE HERE END
            {"role": "user", "content": description},
        ],
        temperature=0.0,
    )
    message = response.content
    # YOUR CODE HERE START: Parse the response
    return message.replace(".", "").strip()
    # YOUR CODE HERE END

# Here we test your function against the known correct answers
# Feel free to inspect df["neighborhood"] to get a better understanding go the answers.
for i, row in df.iterrows():
    pets_allowed = are_pets_allowed(row["description"])
    assert pets_allowed in ["allowed", "not_allowed", "unknown"], f"For row {i}, we got unexpected class: {pets_allowed}"
    assert pets_allowed == row["pets_allowed"], f"For row {i}, we expected {row['pets_allowed']} but got {pets_allowed}"
    print(f"✅ Row {i} passed {pets_allowed} == {row['pets_allowed']}")


✅ Row 0 passed allowed == allowed
✅ Row 1 passed unknown == unknown
✅ Row 2 passed not_allowed == not_allowed
✅ Row 3 passed not_allowed == not_allowed
✅ Row 4 passed unknown == unknown
✅ Row 5 passed not_allowed == not_allowed
✅ Row 6 passed unknown == unknown
✅ Row 7 passed unknown == unknown
✅ Row 8 passed unknown == unknown
✅ Row 9 passed unknown == unknown


## Exercise 1e: Reflection
So far, we have implemented multiple functions that each extract some information from the description.
Before we continue, let's reflect on the current approach.
- What are the disadvantages of the current approach with separate functions for each extraction task? 
- How could we improve the current approach?

## Optional exercise 1f: Combine all the extraction methods into a single prompt
Let's try to combine all the extraction methods into a single function.
It is your task to write a prompt that extracts this information from the description in the function below.

For example:
- Description: "This is a 3 bedrooms house in De Pijp where pets are not allowed."
- Extracted: `{"n_bedrooms": 3, "neighborhood": "De Pijp", "pets_allowed": "not_allowed"}`

If you run this cell, it will automatically test your function against the known correct answers in the dataset.

Note: might need the [json.load](https://docs.python.org/3/library/json.html) function to parse a JSON string into a Python dictionary.

Note: In the Chat API, setting `response_format={"type": "json_object"}` guarantees the ouput is a JSON (although it doesn't guarantee it will be the format we want!).

In [77]:
import json
from typing import Any
def extract_info(description: str) -> dict[str, Any]:
    # YOUR CODE HERE START
    system_prompt="Extract the number of bedrooms, neighborhood, pets allowed from given house description. The output pets allowed should be either allowed, not_allowed, or unknown. Output should be in JSON."
    # YOUR CODE HERE END
    response = client.invoke(
        input=[
            {"role": "system", "content": system_prompt},
            # YOUR CODE HERE START: Optionally add some few shot examples here
            { "role": "user", "content": """Conditions: - Pets not allowed. The rental price of this house is exclusive Gas/Electricity/Water, TV/Internet and local Taxes.",Amsterdam,2500,78,Kinkerbuurt,2,Appartement,not_allowed "On this beautiful location in the Geuzenbuurt, situated between Oud-West and de Baarsjes and conveniently close to the city center. The apartment has a very practical layout with 2 bedrooms, a separate toilet, a spacious bathroom.""" },
            {"role": "system", "content": "{ 'n_bedrooms': '2', 'neighborhood': 'Kinkerbuurt', 'pets_allowed': 'not_allowed' }" },
            # YOUR CODE HERE END
            {"role": "user", "content": description},
        ],
        # Optional: some models have a JSON response format which allows you to 
        # enforce the response format. Uncomment the following lines if you want to use it.
        # response_format={"type": "json_object"}, # OpenAI or Azure
        response_mime_type="application/json", # GCP
        temperature=0.0,
    )
    
    # YOUR CODE HERE START: Parse the response
    data = json.loads(response.content)
    print(data)
    # YOUR CODE HERE END

    # for every value, attempt to cast ints. Keep original value if it fails.
    for key in data:
        try:
            if key == "n_bedrooms":
                data[key] = int(data[key])
        except ValueError:
            pass

    
    return data

for i, row in df.iterrows():
    info = extract_info(row["description"])
    assert info["n_bedrooms"] == row["bedrooms"], f"For row {i}, we expected {row['bedrooms']} but got {info['n_bedrooms']}"
    assert info["neighborhood"].lower().strip() == row["neighborhood"].lower().strip(), f"For row {i}, we expected `{row['neighborhood']}` but got `{info['neighborhood']}`"
    assert info["pets_allowed"] == row["pets_allowed"], f"For row {i}, we expected {row['pets_allowed']} but got {info['pets_allowed']}"
    print(f"✅ Row {i} passed {info}")

{'n_bedrooms': '1', 'neighborhood': 'Oud West', 'pets_allowed': 'allowed'}


AssertionError: For row 0, we expected `Da Costabuurt` but got `Oud West`

---